In [5]:
pip install torch torchvision timm numpy scikit-learn pandas


In [6]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder  # ✅ Correct Import



In [7]:
# 1. Load ResNet50 Model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 1),
    nn.Sigmoid()
)



In [8]:
# 2. Prepare Dataset
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_data = ImageFolder(r"C:\Users\golla\Desktop\BTECH\8THSEM\train", transform=train_transform)
valid_data = ImageFolder(r"C:\Users\golla\Desktop\BTECH\8THSEM\valid", transform=train_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)



In [9]:
# 3. Training Setup
criterion = nn.BCELoss()  # Binary Cross-Entropy loss for Sigmoid
optimizer = optim.Adam(model.parameters(), lr=0.0001)



In [10]:
# 4. Train Model
num_epochs = 10  # Increase this for better accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().view(-1, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.273952616329648
Epoch 2/10, Loss: 0.054618826438264365
Epoch 3/10, Loss: 0.08078721645002238
Epoch 4/10, Loss: 0.06593798500028226
Epoch 5/10, Loss: 0.07389356798018541
Epoch 6/10, Loss: 0.08060025019375117
Epoch 7/10, Loss: 0.09317628139417553
Epoch 8/10, Loss: 0.06727614996979307
Epoch 9/10, Loss: 0.05363817635939963
Epoch 10/10, Loss: 0.07238251991619013


In [11]:
# 5. Save Trained Model

torch.save(model.state_dict(), "fake_news_model_resnet50.pth")
print("✅ Model trained and saved as 'fake_news_model_resnet50.pth'")


✅ Model trained and saved as 'fake_news_model_resnet50.pth'


In [25]:
def predict_image(image_path, threshold=0.4):  # Adjust threshold
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)

    probability = output.item()  # Sigmoid probability

    label = "Real News" if probability >= threshold else "Fake News"
    print(f"Prediction: {label}")

# Test with different thresholds
predict_image(r"C:\Users\golla\Desktop\real.jpg", threshold=0.4)  


Prediction: Real News


In [1]:
def predict_image(image_path, threshold=0.4):  # Adjust threshold
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)

    probability = output.item()  # Sigmoid probability

    label = "Real News" if probability >= threshold else "Fake News"
    print(f"Prediction: {label}")

# Test with different thresholds
predict_image(r"C:\Users\golla\Desktop\fake.jpg", threshold=0.4)  


Prediction: Fake News


In [17]:

# 6. Test Model on Test Data

model.eval()
test_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        test_correct += (outputs.argmax(1) == labels).sum().item()
        total_samples += labels.size(0)

test_acc = 100 * test_correct / total_samples
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 92.59%
